<a href="https://colab.research.google.com/github/nadiajelani/football_analytics/blob/main/deform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install OpenCV if not already installed
!pip install opencv-python-headless


In [2]:
# Import libraries
import cv2
import numpy as np
import os
from google.colab import drive  # Import drive to mount Google Drive
# Mount Google Drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
image_folder = '/content/drive/MyDrive/Data and Videos/Ball 1/Drop 1'

In [ ]:
# List to store image file names (assuming sequential names like frame1.bmp, frame2.bmp, etc.)
image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.bmp')])
if not image_files:
    raise ValueError(f"No .bmp files found in {image_folder}. Please check the folder path and ensure .bmp files exist.")

# Read the first image to initialize
frame = cv2.imread(image_files[0], cv2.IMREAD_GRAYSCALE)
if frame is None:
    raise ValueError(f"Failed to load the first image: {image_files[0]}")

In [ ]:
# Preprocess the first frame
_, thresh = cv2.threshold(frame, 50, 255, cv2.THRESH_BINARY_INV)

# Find contours to get initial object position
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    M = cv2.moments(largest_contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
    else:
        cx, cy = frame.shape[1] // 2, frame.shape[0] // 2  # Default center if no contour
else:
    cx, cy = frame.shape[1] // 2, frame.shape[0] // 2

# Initialize Kalman Filter (4 states: x, y, dx, dy; 2 measurements: x, y)
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
kalman.statePre = np.array([[cx], [cy], [0], [0]], np.float32)  # Initial state

In [ ]:
# Apply Canny edge detection to find the surface line
edges = cv2.Canny(frame, 50, 150, apertureSize=3)

# Use Hough Transform to detect lines
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

# Find the horizontal line closest to the bottom (assuming it's the surface)
surface_y = frame.shape[0]  # Default to bottom if no line is found
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if abs(y1 - y2) < 10:  # Horizontal line (small y difference)
            surface_y = min(y1, y2)  # Take the lowest y as the surface
            break

print(f"Detected surface at y = {surface_y}")

In [ ]:
# Import the necessary patch
from google.colab.patches import cv2_imshow
# Initialize previous frame for optical flow
prev_gray = cv2.cvtColor(cv2.imread(image_files[0]), cv2.COLOR_BGR2GRAY)
p0 = np.array([[[cx, cy]]], dtype=np.float32)  # Initial point for optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Process each image
for i, img_file in enumerate(image_files):
    # Read and preprocess current frame
    frame = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    if frame is None:
        print(f"Failed to load {img_file}")
        continue

    _, thresh = cv2.threshold(frame, 50, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cx, cy = None, None  # Reset for this frame
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest_contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            # Update Kalman filter with measurement
            kalman.correct(np.array([[np.float32(cx)], [np.float32(cy)]]))

    # Predict with Kalman filter (handles occlusion)
    prediction = kalman.predict()
    pred_cx, pred_cy = int(prediction[0][0]), int(prediction[1][0])  # Fixed indexing

    # Optical flow (optional confirmation)
    curr_gray = frame
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, p0, None, **lk_params)
    if st[0] == 1:
        flow_cx, flow_cy = p1[0][0], p1[0][1]
    else:
        flow_cx, flow_cy = pred_cx, pred_cy
    p0 = np.array([[flow_cx, flow_cy]], dtype=np.float32)

    # Convert frame to color for drawing
    frame_color = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)

    # Draw predicted and flow positions
    cv2.circle(frame_color, (pred_cx, pred_cy), 5, (0, 0, 255), -1)  # Kalman prediction
    cv2.circle(frame_color, (int(flow_cx), int(flow_cy)), 5, (0, 255, 0), -1)  # Optical flow

    if contours:
        # Draw the contour
        cv2.drawContours(frame_color, [largest_contour], -1, (255, 0, 0), 2)

        # Get bounding box to find bottom edge
        x, y, w, h = cv2.boundingRect(largest_contour)
        bottom_y = y + h

        # Detect contact
        if bottom_y >= surface_y:
            print(f"Contact detected at frame {i + 1} ({img_file})")

            # Measure contact length (width of the contour at the surface)
            contact_points = largest_contour[np.abs(largest_contour[:, 0, 1] - surface_y) < 5]  # Fixed indexing
            if len(contact_points) > 0:
                contact_length = np.max(contact_points[:, 0, 0]) - np.min(contact_points[:, 0, 0])  # Fixed indexing
                print(f"Contact length: {contact_length} pixels")

            # Measure deformation (aspect ratio of bounding box)
            aspect_ratio = w / h
            print(f"Deformation (aspect ratio): {aspect_ratio}")

    # Draw surface line
    cv2.line(frame_color, (0, surface_y), (frame.shape[1], surface_y), (255, 255, 0), 2)

    # Display frame
    cv2_imshow(frame_color)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Update previous frame for optical flow
    prev_gray = curr_gray.copy()

# Release and close
cv2.destroyAllWindows()